In [1]:
import pickle
import numpy
import pandas as pd
from onnxruntime import InferenceSession
import onnxruntime as rt
from sklearn.base import ClassifierMixin
from lightgbm import LGBMClassifier, LGBMRegressor, Dataset, train, Booster
from skl2onnx import update_registered_converter
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx.common.shape_calculator import (
    calculate_linear_classifier_output_shapes,  # noqa
    calculate_linear_regressor_output_shapes,
)
from onnxmltools.convert.lightgbm.operator_converters.LightGbm import (
    convert_lightgbm  # noqa
)
import onnxmltools
from onnxmltools.convert.lightgbm._parse import WrappedBooster  # noqa
from skl2onnx import to_onnx
from skl2onnx._parse import (
    _parse_sklearn_classifier, _parse_sklearn_simple_model)

In [2]:
def calculate_lightgbm_output_shapes(operator):
    op = operator.raw_operator
    if hasattr(op, "_model_dict"):
        objective = op._model_dict['objective']
    elif hasattr(op, 'objective_'):
        objective = op.objective_
    else:
        raise RuntimeError(  # pragma: no cover
            "Unable to find attributes '_model_dict' or 'objective_' in "
            "instance of type %r (list of attributes=%r)." % (
                type(op), dir(op)))
    if objective.startswith('binary') or objective.startswith('multiclass'):
        return calculate_linear_classifier_output_shapes(operator)
    if objective.startswith('regression'):  # pragma: no cover
        return calculate_linear_regressor_output_shapes(operator)
    raise NotImplementedError(  # pragma: no cover
        "Objective '{}' is not implemented yet.".format(objective))

In [3]:
def lightgbm_parser(scope, model, inputs, custom_parsers=None):
    if hasattr(model, "fit"):
        raise TypeError(  # pragma: no cover
            "This converter does not apply on type '{}'."
            "".format(type(model)))

    if len(inputs) == 1:
        wrapped = WrappedBooster(model)
        objective = wrapped.get_objective()
        if objective.startswith('binary'):
            wrapped = WrappedLightGbmBoosterClassifier(wrapped)
            return _parse_sklearn_classifier(
                scope, wrapped, inputs, custom_parsers=custom_parsers)
        if objective.startswith('multiclass'):
            wrapped = WrappedLightGbmBoosterClassifier(wrapped)
            return _parse_sklearn_classifier(
                scope, wrapped, inputs, custom_parsers=custom_parsers)
        if objective.startswith('regression'):  # pragma: no cover
            return _parse_sklearn_simple_model(
                scope, wrapped, inputs, custom_parsers=custom_parsers)
        raise NotImplementedError(  # pragma: no cover
            "Objective '{}' is not implemented yet.".format(objective))

    # Multiple columns
    this_operator = scope.declare_local_operator('LightGBMConcat')
    this_operator.raw_operator = model
    this_operator.inputs = inputs
    var = scope.declare_local_variable(
        'Xlgbm', inputs[0].type.__class__([None, None]))
    this_operator.outputs.append(var)
    return lightgbm_parser(scope, model, this_operator.outputs,
                           custom_parsers=custom_parsers)

In [4]:
class WrappedLightGbmBoosterClassifier(ClassifierMixin):
    """
    Trick to wrap a LGBMClassifier into a class.
    """

    def __init__(self, wrapped):  # pylint: disable=W0231
        for k in {'boosting_type', '_model_dict', '_model_dict_info',
                  'operator_name', 'classes_', 'booster_', 'n_features_',
                  'objective_', 'boosting_type', 'n_features_'}:
            if hasattr(wrapped, k):
                setattr(self, k, getattr(wrapped, k))

In [7]:
lgm_model = Booster(model_file='model.txt')
with open('model.pkl', 'wb') as f:
    pickle.dump(lgm_model, f)

In [8]:
with open('model.pkl', "rb") as mp:
    lgbm_model: LGBMClassifier = pickle.load(file=mp) 


In [9]:
lgbm_model.num_feature()

2021

In [10]:
update_registered_converter(
            WrappedLightGbmBoosterClassifier,
            'WrappedLightGbmBoosterClassifier',
            calculate_lightgbm_output_shapes,
            convert_lightgbm, parser=lightgbm_parser,
            options={'zipmap': [False, True], 'nocl': [False, True]})
update_registered_converter(
            WrappedBooster, 'WrappedBooster',
            calculate_lightgbm_output_shapes,
            convert_lightgbm, parser=lightgbm_parser,
            options={'zipmap': [False, True], 'nocl': [False, True]})
update_registered_converter(
            Booster, 'LightGbmBooster', calculate_lightgbm_output_shapes,
            convert_lightgbm, parser=lightgbm_parser)

In [11]:
update_registered_converter(
            WrappedLightGbmBoosterClassifier,
            'WrappedLightGbmBoosterClassifier',
            calculate_lightgbm_output_shapes,
            convert_lightgbm, parser=lightgbm_parser,
            options={'zipmap': [False, True], 'nocl': [False, True]})

In [12]:
onnx_model = to_onnx(lgbm_model, initial_types=[('feature_input', FloatTensorType([None, 2021]))],
            options={WrappedLightGbmBoosterClassifier: {'zipmap': False}}, target_opset={'': 15, 'ai.onnx.ml': 2})